https://www.nature.com/articles/s41598-019-45605-1

https://www.frontiersin.org/articles/10.3389/fnhum.2020.00338/full

https://backend.orbit.dtu.dk/ws/files/153136597/1570435270.pdf

https://arxiv.org/pdf/2004.00077.pdf


https://www.mdpi.com/1424-8220/21/19/6570

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Conv2D, DepthwiseConv2D, AveragePooling2D, Activation, SeparableConv2D, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import glob
import numpy as np
from utils import preprocess_data
from mne.preprocessing import ICA

2023-09-04 20:56:21.341300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:

'''
=========  ===================================
run        task
=========  ===================================
1          Baseline, eyes open
2          Baseline, eyes closed
3, 7, 11   Motor execution: left vs right hand
4, 8, 12   Motor imagery: left vs right hand
5, 9, 13   Motor execution: hands vs feet
6, 10, 14  Motor imagery: hands vs feet
=========  ===================================
'''
raws_train = []
raws_test = []
for ii in range(1, 15):
    subject = f'S{ii:03d}'
    files = glob.glob(f'../files/{subject}/*.edf')
    for i in [5, 9, 13, 3, 7, 11]:
        current_file = files[i]
        r = read_raw_edf(current_file, preload=True, stim_channel='auto')
        events, _ = mne.events_from_annotations(r)
        if i in [5, 9, 13]:
            new_labels_events = {1:'rest', 2:'T1', 3:'T2'} # action
        elif i in [3, 7, 11]:
            new_labels_events = {1:'rest', 2:'T3', 3:'T4'}
        new_annot = mne.annotations_from_events(events=events, event_desc=new_labels_events, sfreq=r.info['sfreq'], orig_time=r.info['meas_date'])
        r.set_annotations(new_annot)
        if ii < 4:
            raws_test.append(r)
        else:
            raws_train.append(r)
    
raws_train_obj = concatenate_raws(raws_train)
raw_test_obj = concatenate_raws(raws_test)

Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/fil

In [7]:

raw = raw_obj.copy()

# "Fc5.","Fc6.","Fc3.","Fc4.","Fc1.","Fc2.","C5..","C6..","C3..","C4..","C1..","C2..","Cp5.","Cp6.","Cp3.","Cp4.","Cp1.","Cp2."
raw = raw.pick_channels(['Fc5.', 'Fc6.', 'Fc3.', 'Fc4.', 'Fc1.', 'Fc2.', 'C5..', 'C6..', 'C3..', 'C4..', 'C1..', 'C2..', 'Cp5.', 'Cp6.', 'Cp3.', 'Cp4.', 'Cp1.', 'Cp2.'])

# filters
notch_freq = 60
raw.notch_filter(notch_freq, fir_design='firwin')

low_cutoff = 8
high_cutoff = 40
raw.filter(low_cutoff, high_cutoff, fir_design='firwin')

events, event_dict = mne.events_from_annotations(raw)
print(raw.info)
print(event_dict)
picks = mne.pick_types(raw.info, meg=True, eeg=True, stim=False, eog=False, exclude='bads')
# eegbci.standardize(raw)
# montage = mne.channels.make_standard_montage('standard_1005')
# raw.set_montage(montage)
# ## ICA
# n_components = 10
# ica = ICA(n_components=n_components, random_state=97, max_iter=800)
# ica.fit(raw)
# components_to_excludes, scores = ica.find_bads_eog(raw, ch_name='Fpz')
# if components_to_excludes is not None and len(components_to_excludes) > 0:
#     ica.exclude = components_to_excludes
#     raw = ica.apply(raw)
# else:
#     print("No components to exclude")

event_id = {'T1': 2, 'T2': 3, 'T3': 4, 'T4': 5}
events, event_dict = mne.events_from_annotations(raw, event_id=event_id)
tmin = -2  # Time before event in seconds
tmax = 2.  # Time after event in seconds
epochs = mne.Epochs(raw, events, event_dict, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

# raw, events, event_dict, picks, epochs

In [56]:
event_id = {'T1': 2, 'T2': 3, 'T3': 4, 'T4': 5}
events, event_dict = mne.events_from_annotations(raw, event_id=event_id)
event_dict

Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4']


{'T1': 2, 'T2': 3, 'T3': 4, 'T4': 5}

In [57]:
epochs.get_data().shape

(735, 18, 641)

In [58]:
y = epochs.events[:, -1] - 2
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

{0: 197, 1: 193, 2: 173, 3: 172}


In [59]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs.get_data(), epochs.events[:, -1]- 2, test_size=0.4, random_state=42)
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)
print(X_train.shape, y_train.shape)

(441, 18, 641) (441, 4)


In [64]:
# Load and preprocess EEG data
# Assuming you have EEG_data and labels




# # Create 1D CNN -> LSTM model
# model = Sequential()
# model.add(Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=88, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.5))
# model.add(LSTM(256, return_sequences=True))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(64))
# model.add(Dense(1, activation='softmax'))


model = Sequential()
# model.add(Conv1D(25, 11, strides=1, input_shape=(X_train.shape[1], X_train.shape[2]), padding='valid', activation='relu'))
# model.add(Conv1D(25, 2, strides=1, padding='valid'))
# model.add(Conv1D(25, 2, strides=1, padding='valid'))
# # Max Pooling Layer 1
# model.add(MaxPooling1D(pool_size=3, strides=3, padding='same'))

# # Convolutional Layer 3
# model.add(Conv1D(50, 11, strides=1, padding='same', activation='relu'))

# # Max Pooling Layer 2
# model.add(MaxPooling1D(pool_size=3, strides=3, padding='same'))

# # Convolutional Layer 4
# model.add(Conv1D(100, 11, strides=1, padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# # Max Pooling Layer 3
# model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))

# # Convolutional Layer 5
# model.add(Conv1D(200, 11, strides=1, padding='same', activation='relu'))

# # Max Pooling Layer 4
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))

model.add(Conv1D(filters=640, kernel_size=11, strides=1, activation='elu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Conv1D(filters=630, kernel_size=11, strides=1, activation='elu', padding='same'))

# model.add(MaxPooling1D(pool_size=3))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=210, kernel_size=50, strides=1, activation='elu', padding='same'))
model.add(SpatialDropout1D(0.5))
# model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(filters=64, kernel_size=100, strides=1, activation='elu', padding='same'))
# # normalize
model.add(BatchNormalization())
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=22, kernel_size=200, strides=1, activation='elu', padding='same'))
model.add(BatchNormalization())
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=8, kernel_size=300, strides=1, activation='elu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(SpatialDropout1D(0.5))

model.add(MaxPooling1D(pool_size=2))
# model.add(MaxPooling1D(pool_size=2))
# model.add(MaxPooling1D(pool_size=2))
# flatten
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# 4 classes output
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Epoch 1/100
118/118 [==============================] - 32s 238ms/step - loss: 1.5985 - accuracy: 0.2614 - val_loss: 1.3899 - val_accuracy: 0.2360
Epoch 2/100
118/118 [==============================] - 28s 239ms/step - loss: 1.4682 - accuracy: 0.2699 - val_loss: 1.3971 - val_accuracy: 0.2360
Epoch 3/100
118/118 [==============================] - 28s 234ms/step - loss: 1.4742 - accuracy: 0.2812 - val_loss: 1.4488 - val_accuracy: 0.2472
Epoch 4/100
118/118 [==============================] - 28s 234ms/step - loss: 1.4812 - accuracy: 0.2528 - val_loss: 1.3930 - val_accuracy: 0.2472
Epoch 5/100
118/118 [==============================] - 28s 235ms/step - loss: 1.4661 - accuracy: 0.2443 - val_loss: 1.3978 - val_accuracy: 0.2472
Epoch 6/100
118/118 [==============================] - 28s 236ms/step - loss: 1.4507 - accuracy: 0.2585 - val_loss: 1.4002 - val_accuracy: 0.2472
Epoch 7/100
118/118 [==============================] - 28s 236ms/step - loss: 1.4689 - accuracy: 0.2188 - val_loss: 1.3872 -

In [52]:
# Load and preprocess EEG data
# Assuming you have EEG_data and labels




# # Create 1D CNN -> LSTM model
# model = Sequential()
# model.add(Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=88, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.5))
# model.add(LSTM(256, return_sequences=True))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(64))
# model.add(Dense(1, activation='softmax'))

Nf = X_train.shape[2]
Ncl = len(np.unique(y))
Nch = 64
Ns = X_train.shape[0]

model = Sequential()
model.add(Conv2D(8, kernel_size=(Nf, 1), strides=(1, 1), padding='same', input_shape=(Ns, Nch, 8)))
model.add(BatchNormalization())

# Layer φ2: DepthwiseConv2D
model.add(DepthwiseConv2D(kernel_size=(1, Nch), strides=(1, 1), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=(8, 1), padding='valid'))

# Layer φ3: SeparableConv2D
model.add(SeparableConv2D(16, kernel_size=(16, 1), strides=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=(8, 1), padding='valid'))

# Flatten and fully connected layer (φ4)
model.add(Flatten())
model.add(Dropout(0.5))
# 4 classes output
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=3, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Epoch 1/10


ValueError: in user code:

    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/si

In [49]:
X_train.shape

(441, 64, 641)